In lecture 3 we saw how gaussian inference turns into linear-algebra.

In lecture 4 we'll see how linear algebra <i>is</i> gaussian inference.

why is this useful to know?

Given $p(x) = \mathcal{N}(x; \mu, \Sigma)$ then \
$p(x \mid y = \mathcal{N}(x; \mu', \Sigma')$ \
here $\mu'$ and $\Sigma'$ are expressed in terms of `Precision Matrix` or `Gain + Residual + Gram`. This is the most basic machine-learning architecture. It has observations and unkowns which are related by a linear model $y = Ax + b$ 

what does PyTorch DataLoader do? its a piece of code that goes through the dataset and randomly picks batches of data. why? why not just specific data points? why batches? in linear algebra there are precise answers to these questions.



Observations are described by a linear transformation of the distribution variable. \
Measurements can change with time. \
For $n$ observations, there could be $n$ separate affine-transformations $A_{i...n}, \vec{b}_{i...n}$ \
$A^T\vec{x} = \vec{b} \quad $  where $A \in \mathbb{R}^{M \times N}$, $ \vec{b} \in \mathbb{R}^N , \vec{x}  \in \mathbb{R}^M$ \
Assume $A$ has full-rank (i.e all independant columns) \
Assume $N \leq M$ i.e we have more unknown parameters than data points. \
Find $\vec{x} \in \mathbb{R}^{M}$ the distribution of parameters that _best_ descirbes the data points 

#### Gram-Schmidt
The Gram-Schmidt process is a method for turning a set of linearly independent vectors into an orthonormal set that spans the same subspace.

Given a span of vectors $\vec{v_i}$ we need to produce a span of orthonormal vectors $\vec{u}_i$. \
The idea is to subtract the projection of $\vec{v}_i\cdot \vec{v}_{i+1}$ from $v_{i+1}$ from other 

1. $\vec{u}_0 = \vec{v}_0/\lVert \vec{v}_0 \rVert$
2. $\vec{u}_1 = \vec{w}_1/\lVert \vec{w}_1 \rVert$  where $\vec{w}_1 = \vec{v}_1 - (\vec{v}_1\cdot \vec{u}_0)\vec{u}_0$
3. $\vec{u}_2 = \vec{w}_2/\lVert \vec{w}_2 \rVert$  where $\vec{w}_2 = \vec{v}_2 - (\vec{v}_2 \cdot \vec{u}_1) \vec{u}_1 - (\vec{v}_1 \cdot \vec{u}_0 ) \vec{u}_0$

#### Singular Value Decomposition

we want to solve $A\vec{x} = \vec{b}$ where $A \in \mathbb{R}^{M \times N}, \vec{b} \in \mathbb{R}^M, \vec{x} \in \mathbb{R}^N$ 

And we want to consider 3 possible case 

1. $A$ is square, full rank i.e $M = N$. SVD can solve this exactly. 
2. Overdetermined i.e $M > N$. SVD can find $\vec{x}$ that minimizes $\lVert A \vec{x} - \vec{b}\rVert^2$. 
3. Underdetermined i.e $M<N$ there are infinitely many solutions for $\vec{x}$ that satisfies $A\vec{x} = \vec{b}$ but SVD will give use the __minimum-norm solution__ i.e the $\vec{x}$ with minimum $\lVert \vec{x} \rVert ^2$

#### How SVD works
The SVD decomposes $A = Q D U^T$ where 
- $Q \in \mathbb{R}^{M \times M} $ is orthogonal
- $ D \in \mathbb{R}^{M \times N} $ is diagonal
- $ U \in \mathbb{R}^{N \times N}$ is orthogonal

Now, solving for $\vec{x} = A^{-1}\vec{b}$ is just $\vec{x} = U D^{-1} Q^T \cdot \vec{b}$ and since $D$ is diagonal, the $D^{-1}$ is just element-wise reciprocal. 

In [10]:

# The inverse of A is implemented as a standalone method here 
# because computing the inverse of A is the most time-consuming part of 
# using SVD to solve linear equations

import numpy as np

A = np.array([[1,2,3],[4,5,6],[7,8,9]])
b1 = np.array([9.1,2,3])
b2 = np.array([0.1,2,3])
b3 = np.array([1,2,3])

def solver(A): # for A in M x N
    Q,D,Ut = np.linalg.svd(A.T, full_matrices=False)
    return lambda b: Ut.T @ ((Q.T @ b) / D)

slv = solver(A)
x1 = slv(b1)
x2 = slv(b2)
x3 = slv(b3)

print(x1)
print(x2)
print(x3)

[-2.67900287e+16  5.35800573e+16 -2.67900287e+16]
[ 2.97666985e+15 -5.95333970e+15  2.97666985e+15]
[-0.96555419  3.93110838 -1.96555419]


now put these ideas aside and imagine you've just had undergrad linear algebra. you know about matrices and inverses but you dont know about SVD. And you're encountering the problerm of evaluating the posterior given a series of observations. 

$p(z \mid B^Tz = y + \epsilon) = \mathcal{N}(z; m + \Delta m, V - \Delta V)$ where \
$\Delta m = VB(B^TVB + \Lambda)^{-1}(y - B^Tm)$ \
$\Delta V = VB(B^TVB + \Lambda)^{-1}B^TV$

This is actually an imoprtant lecture. I dont fully understand what he's trying to do. but he has an algorithm for computing $p(X \mid y_1,...y_n)$ i.e the posterior given some observations. But, his algorithm allows you to stop at any $ i\leq n$. So that's a computational advantage. I might understand it if i came back to it at a later time

#### Here's my attempt at trying to find an expression for $P(x|y_1,...y_n)$

$P(x) \sim \mathcal{N}(x;\mu_0,\Sigma_0)$ \
$y_i = A_i \cdot x + b_i + \Lambda $ where $\Lambda \sim \mathcal{N}(0,\lambda^2)$ is the measurement noise

$P(x|y_1) = \mathcal{N}(x, \mu_1, \Sigma_1)$ where $\mu_1,\Sigma_1$ are expressed in terms of the `Precision Matrix` or `Gain * Residue` 

Then $P(x|y_1,y_2) = \frac{P(y_1 y_2 | x) * P(x)}{P(y_1 y_2)}$ 

Then $P(x|y_1,y_2) = \frac{P(y_1 y_2 | x) * P(x)}{\int P(y_1 y_2 | x)dx}$

$P(y_1 y_2 | x) = P(y_1 | x)P(y_2 | x)$ because $y_1,y_2$ are independant measurements when fixed on a conditional $x$ 

$P(y_1|x)$ and $P(y_2|x)$ are both gaussians with the same variance as $\Lambda$ with just the mean shifted to $A_i x + b_i$

So this can be calculated analytically, complete the squares, you get one big ugly gaussian for $P(x|y_1,y_2)$

But the trick is to express $P(x|y_1,y_2)$ in terms of $\mu_1$ and $\Sigma_1$ that we computed for $P(x|y_1)$



### And here's how he does it 
_he considers the case where the observation are noiseless, scalar and the transofrmation $A_i$ is just a vector_ i.e $y_i = a_i.x$

Computing $P(x|y_1)$ gives us \
$\Sigma_1 = \Sigma_0 - \Sigma_0 A_1 (A_1^T \Sigma_0 A_1 + \Lambda)^{-1}(A_1^T \Sigma_0)$ 

Now use $P(x|y_1)$ as the __prior__ for the $P(x|y_1 y_2)$. I'm not sure what the correct notation for it would be - perhaps $P((x|y_1) | y_2)$. Then we get 

$\Sigma_2 = \Sigma_1 - \Sigma_1 A_2 (A_2^T \Sigma_1 A_2 + \Lambda)^{-1}(A_2^T \Sigma_1)$ 

More generally: \
$\Sigma_i = \Sigma_{i-1} - u_{i}\cdot u_{i}^T$ \
$\Sigma_i = \Sigma_0 - \lbrack u_1\cdot u_1^T + ... + u_i\cdot u_i^T \rbrack$ \
where $u_i = \frac{1}{\sqrt{A_i^T \Sigma_{i-1} A_i}}\cdot \Sigma_{i-1} A_i$ \
and $u_i \Sigma_0^{-1}u_j = \delta_{i,j}$ \
${u_i}$ is orthonormal under transformation of $\Sigma_0$ \
$\Sigma_0$ is a space that spans the original prior distribution of $P(x)$ \ 
$A_i$ is just a vector so $u_i \cdot u_i^T$ is an outer-product not an inner-product. \
_not sure how this generalizes to matrix-transofrmations for observations. \
also he assumes noise-less measurements i.e $\Lambda = 0$ so I'm not sure if the orthogonality of ${u_i}$ would still be valid in the noisy measurements_

This is just rediscovering the __Gram-Schmidt Process__